In [3]:
#from pip._internal import main as pip_main
#except ImportError:
#    from pip._internal import main as pip_main

#package_to_install = "numpy"
#try:
#    pip_main(["install", "--user", package_to_install])
#except SystemExit as e:
#    print(e)

The Newtonian Gravitational Force depends upon the masses of the two objects and the square of the distances between them. When one astronomical body acts upon another with force F12, the second body acts upon the first with equal and opposite force F21=-F12. Gravitational Force follows an inverse square law with regard to distance scaling that is exact in Newtonian gravity.

In [4]:
import numpy as np
import matplotlib.pyplot as plt

def NewtonianForce(mass1,mass2, r):
    return (6.67408*10**-11)*mass1*mass2/r/r;

To compute an orbit, it is necessary to solve a second order differential equation. 

F=ma

For a two body problem:

F12x=m1 d^2 x1/dt^2

This can be decomposed into two coupled differential equations.

F12x= m1 d v1/dt

v1= d x1/dt

There is also a y component and a z component. 
It is also necessary to track the position of the second particle.

Recall that once a force F12 is split into its components in the x and y direction, F12 will no longer exactly be given by the NewtonainForce equation given above but rather the magnitude of F12x, F12y, and F12z combined will equal that total. The nagnitude of a vector is found by squaring and summing the components then taking the square roots. So, sqrt(F12x^2+F12y^2+F12z^2)=FNewtonianForce

Assume, for the purposes of a two body problem, the orbit is confined to the x-y plane. 

Define polar coordinates phi in the x-y plane beginning from the x axis and proceeding toward the y axis. Then F12x=F12 cos phi and F12y= F12 sin phi.

F12x=x/sqrt(x^2+y^2)*F12

F12y = y/sqrt(x^2+y^2)*F12

# Numerical solutions to differential equations

To integrate a differential equation numerically, simply differentiate both of the two coupled equations, one time steop at a time, numerically. But how is this done numerically? There are a couple of ways. First consider how a derivative is defined. A derivative is a limit of a difference divided by a timestep. The euler derivative makes this definition discrete for small time steps. 

In [5]:
def euler(h,t, x,y,z,f,debugprint):
    xstep = (f(t, x, y,z)+f(t+h,x+h,y,z))/2*h
    xnew=x+xstep
    if(debugprint):
        print(h,t,x,xstep,xnew)
    return t+h,xnew

The euler routine essentially just finds the slope in a small region of the graph. 

The Runga Kutta numerical derivative is a fourth order approximation to the derivative that accounts for the curvature and is better for something like an orbit that fundamentally has curvature to it where orvershoot near a minima may matter. 

# Verify Euler numerically

In [31]:
def polynomial(x,power,a):
    p=a*x**power
    #print(pi)
    return p
    

def polymaker1(t,x,y,z,power,a):
    def poly(t,x,y,z):
        return polynomial(x,power,a)
    return poly

#exact solution to pow=1 is lnx

In [33]:
import numpy as np
t=0.
x=1.
y=1.
z=1.
print(t,x)
polyfn=polymaker1(t,x,y,z,1,1.0)

0.0 1.0


In [8]:
outp=polyfn(t,x,y,z) #this is correct: x^2
print(outp)

1.0


In [9]:
eulerpolyout=euler(.01,t,x,y,z,polyfn,True)
print(eulerpolyout)

0.01 0.0 1.0 0.01005 1.01005
(0.01, 1.01005)


In [10]:
dt=0.01
numsteps=10
xvals=[]
tvals=[]
for i in np.arange(1,numsteps):
    #t=0.+i*dt
    t,x=euler(dt,t,x,y,z,polyfn,True)
    print(x)
    xvals.append(x)
    tvals.append(t)
print(xvals)
print(tvals)

0.01 0.0 1.0 0.01005 1.01005
1.01005
0.01 0.01 1.01005 0.0101505 1.0202004999999998
1.0202004999999998
0.01 0.02 1.0202004999999998 0.010252005 1.0304525049999997
1.0304525049999997
0.01 0.03 1.0304525049999997 0.010354525049999999 1.0408070300499996
1.0408070300499996
0.01 0.04 1.0408070300499996 0.010458070300499998 1.0512651003504996
1.0512651003504996
0.01 0.05 1.0512651003504996 0.010562651003504998 1.0618277513540046
1.0618277513540046
0.01 0.060000000000000005 1.0618277513540046 0.010668277513540048 1.0724960288675447
1.0724960288675447
0.01 0.07 1.0724960288675447 0.010774960288675448 1.0832709891562202
1.0832709891562202
0.01 0.08 1.0832709891562202 0.010882709891562202 1.0941536990477825
1.0941536990477825
[1.01005, 1.0202004999999998, 1.0304525049999997, 1.0408070300499996, 1.0512651003504996, 1.0618277513540046, 1.0724960288675447, 1.0832709891562202, 1.0941536990477825]
[0.01, 0.02, 0.03, 0.04, 0.05, 0.060000000000000005, 0.07, 0.08, 0.09]


In [11]:
def analyticsoln1(array):
        returnarray=np.exp(array)
        return returnarray
analyticxvals=analyticsoln1(tvals)
print(analyticxvals)

[1.01005017 1.02020134 1.03045453 1.04081077 1.0512711  1.06183655
 1.07250818 1.08328707 1.09417428]


# Implement RK4

In [12]:
def RK4(h,t,x,y,z,f,printdebug): #not a finite difference so no step in y
    k1= h*f(t,x,y,z)
    k2=h*f(t+h/2.,x+k1/2.,y,z)
    k3=h*f(t+h/2.,x+k2/2.,y,z)
    k4=h*f(t+h,x+k3,y,z)
    if(printdebug):
        print(f(t,x,y,z))
        print(t,x,y,z,h) 
        print("k1",k1)
        print("k2",k2)
        print("k3",k3)
        print("k4",k4)
        print(1./6.*(k1+2.*k2+2.*k3+k4))
        print(x+1/6.*(k1+2.*k2+2.*k3+k4))
    return t+h, x+1/6.*(k1+2.*k2+2.*k3+k4)



In [13]:
def RK4nodebug(h,t,x,y,z,f): #not a finite difference so no step in y
    k1= h*f(t,x,y,z)
    #print("k1",k1)
    k2=h*f(t+h/2.,x+k1/2.,y,z)
    #print("k2",k2)
    k3=h*f(t+h/2.,x+k2/2.,y,z)
    #print("k3",k3)
    k4=h*f(t+h,x+k3,y,z)
    #print("k4",k4)
    return t+h, x+1/6.*(k1+2.*k2+2.*k3+k4)



# Verifying the RK4 routine numerically

First I verify the RK4 by examining the behavior for a differential equation where the right hand side is a fourth order polynomial. This should have a truncation error that is on the order of the fourth order polynomial itself

In [14]:
def polynomial4(t,x,a,b,c,d,e):
    p=a*t**4+b*t**3+c*t**2+d*t+e
    #print(pi)
    return p
    

def polymaker(t,x,y,z,a,b,c,d,e):
    def poly(t,x,y,z):
        return polynomial4(t,x,a,b,c,d,e)
    return poly
        
        

In [15]:
import numpy as np
t=0.
x=1.0
y=0.
z=0.0
print(t,x)
polyfn=polymaker(t,x,y,z,1.,0.,0.,0.,0.)
polyfn1=polymaker1(t,x,y,z,1,-1.0)

0.0 1.0


In [16]:
#import matplotlib.pyplot as plt
#plt.plot(t,polyfn(t,x))

In [17]:
rk4polyout=RK4(.01,t,x,y,z,polyfn1,True)
print(x)
print(outp+x)
print(rk4polyout)

1.0
0.0 1.0 0.0 0.0 0.01
k1 0.01
k2 0.01005
k3 0.01005025
k4 0.010100502499999999
0.010050167083333332
1.0100501670833333
1.0
2.0
(0.01, 1.0100501670833333)


In [18]:
dt=0.01
numsteps=10
intvals=[]
tvals=[]
for i in np.arange(1,numsteps):
    t,x=RK4(dt,t,x,y,z,polyfn1,True)
    #print(valueofintegral)
    intvals.append(x)
    tvals.append(t)
print(intvals)
print(tvals)

1.0
0.0 1.0 0.0 0.0 0.01
k1 0.01
k2 0.01005
k3 0.01005025
k4 0.010100502499999999
0.010050167083333332
1.0100501670833333
1.0100501670833333
0.01 1.0100501670833333 0.0 0.0 0.01
k1 0.010100501670833333
k2 0.0101510041791875
k3 0.010151256691729272
k4 0.010202014237750625
0.01015117294173625
1.0202013400250696
1.0202013400250696
0.02 1.0202013400250696 0.0 0.0 0.01
k1 0.010202013400250697
k2 0.010253023467251949
k3 0.010253278517586955
k4 0.010304546185426566
0.010253193925892513
1.030454533950962
1.030454533950962
0.03 1.030454533950962 0.0 0.0 0.01
k1 0.010304545339509622
k2 0.01035606806620717
k3 0.010356325679840657
k4 0.010408108596308028
0.01035624023798555
1.0408107741889476
1.0408107741889476
0.04 1.0408107741889476 0.0 0.0 0.01
k1 0.010408107741889477
k2 0.010460148280598925
k3 0.010460408483292471
k4 0.010512711826722402
0.010460322182732445
1.05127109637168
1.05127109637168
0.05 1.05127109637168 0.0 0.0 0.01
k1 0.0105127109637168
k2 0.010565274518535384
k3 0.01056553733630947

RK4 is less precise than Euler for a 1st order polynomial RHS

RK4 is precise to 1 part in 10^3 and Euler is precise to 1 part in 10^6 for this equation. 

# Check precision for a fourth order polynomial of RK4 relative to Euler. RK4 should do better 

In [36]:
polyfn4=polymaker1(t,x,y,z,4,-1.0)

In [56]:
dt=0.01
numsteps=30
integralvals=[]
tvals=[]
t=1.
x=(3.)**(-1./3.)
print(x)
for i in np.arange(1,numsteps):
    t,x=RK4(dt,t,x,y,z,polyfn4,True)
    #print(valueofintegral)
    integralvals.append(x)
    tvals.append(t)
print(np.array(integralvals))
print(tvals)
print(len(integralvals),len(tvals))

0.6933612743506348
-0.231120424783545
1.0 0.6933612743506348 1.0 1.0 0.01
k1 -0.00231120424783545
k2 -0.0022958346968050985
k3 -0.0022959366507114584
k4 -0.0022807434751047346
-0.0022959150696622165
0.6910653592809726
-0.22807437600030617
1.01 0.6910653592809726 1.0 1.0 0.01
k1 -0.002280743760003062
k2 -0.002265726568170315
k3 -0.002265825203258446
k4 -0.0022509786648749128
-0.002265804327955916
0.6887995549530167
-0.2250978937754673
1.02 0.6887995549530167 1.0 1.0 0.01
k1 -0.002250978937754673
k2 -0.0022363026773626576
k3 -0.0022363981327168697
k4 -0.00222188704279087
-0.0022363779334507662
0.6865631770195659
-0.22218873042700318
1.03 0.6865631770195659 1.0 1.0 0.01
k1 -0.002221887304270032
k2 -0.0022075410247283976
k3 -0.0022076334323280346
k4 -0.002193447067092411
-0.002207613880912551
0.6843555631386534
-0.2193447317751573
1.04 0.6843555631386534 1.0 1.0 0.01
k1 -0.0021934473177515727
k2 -0.0021794205213487984
k3 -0.002179510006495144
k4 -0.0021656380836222533
-0.002179491076176951

In [57]:
x0=(3.)**(-1./3.)
x0

0.6933612743506348

The analyitic solution to the differential equation

dx/dt = -x^4 

is 

x= (3t)^(-1/3)

Below the output is shown. The results are plotted for scaling comparison. It doesn't look like it scales correctly. '

In [58]:
def ODEsolnanalytic(t):
    return (3.*t)**(-1./3.)

In [59]:
solnanalytic=[]
for i in np.arange(len(tvals)):
    solnanalytic.append(ODEsolnanalytic(tvals[i]))
print(np.array(solnanalytic))
print(len(solnanalytic))

[0.69106536 0.68879955 0.68656318 0.68435556 0.68217607 0.68002408
 0.67789899 0.67580022 0.6737272  0.67167939 0.66965624 0.66765726
 0.66568193 0.66372977 0.66180031 0.6598931  0.65800768 0.65614362
 0.65430051 0.65247794 0.65067551 0.64889282 0.64712952 0.64538523
 0.64365959 0.64195226 0.64026291 0.63859119 0.6369368 ]
29


In [61]:
dt=0.01
numsteps=30
xvals=[]
tvals=[]
for i in np.arange(1,numsteps):
    #t=0.+i*dt
    t,x=euler(dt,t,x,y,z,polyfn4,True)
    print(x)
    xvals.append(x)
    tvals.append(t)
print(xvals)
print(tvals)

0.01 1.3800000000000003 0.622266232609759 -0.0015487209465093541 0.6207175116632496
0.6207175116632496
0.01 1.3900000000000003 0.6207175116632496 -0.0015334852213442186 0.6191840264419054
0.6191840264419054
0.01 1.4000000000000004 0.6191840264419054 -0.001518510420680935 0.6176655160212244
0.6176655160212244
0.01 1.4100000000000004 0.6176655160212244 -0.0015037901981959096 0.6161617258230285
0.6161617258230285
0.01 1.4200000000000004 0.6161617258230285 -0.0014893184069972023 0.6146724074160314
0.6146724074160314
0.01 1.4300000000000004 0.6146724074160314 -0.0014750890919599331 0.6131973183240714
0.6131973183240714
0.01 1.4400000000000004 0.6131973183240714 -0.0014610964824092391 0.6117362218416622
0.6117362218416622
0.01 1.4500000000000004 0.6117362218416622 -0.0014473349851326556 0.6102888868565296
0.6102888868565296
0.01 1.4600000000000004 0.6102888868565296 -0.0014337991777048575 0.6088550876788248
0.6088550876788248
0.01 1.4700000000000004 0.6088550876788248 -0.0014204838021086947 

In [73]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
output_notebook()
# create a new plot with default tools, using figure
p = figure(plot_width=400, plot_height=400)

# add a circle renderer with x and y coordinates, size, color, and alpha
p.circle(tvals,xvals, size=15, line_color="orange",fill_color="orange", fill_alpha=0.5, legend="Euler")
p.circle(tvals,integralvals, size=15, line_color="blue", fill_color="blue", fill_alpha=0.5,legend="RK4")
p.circle(tvals,solnanalytic, size=15, line_color="green",fill_color="green", fill_alpha=0.5,legend="dx/dt=-x^4 exact")


p.circle(tvals,np.array(solnanalytic)-np.array(xvals),size=15, line_color="yellow",fill_color="yellow",fill_alpha=0.5, legend="abs diff Euler")
p.circle(tvals,np.array(solnanalytic)-np.array(integralvals),size=15, line_color="red", fill_color="red", fill_alpha=0.5,legend="abs diff RK4")

p.legend.location = "bottom_left"
p.legend.click_policy="hide"

show(p) # show the results

Loading BokehJS ...

This looks good to me. RK4 performs better on the 4th order RHS ODE than Euler does, they are both converging after initial transients, they both have small truncation error relative to their absolute value which improves with choice of time step, and the truncation error is worse for higher order functions. This seems very reasonable. 

# An equal mass system will be easier to obtain the necessary numerical precision than the Earth-Sun system. 

So start with two stellar mass objects in a binary. 

In [133]:
import math
def InitialDataEqualMass():
    phi=np.array([0,math.pi])
    orbitalangle=np.zeros(2)
    orbitalradius=np.ones(2)
    eccentricity=np.zeros(2)
    mass=np.ones(2)
    masssun=1.989*10**30
    lsun=3.828*10**26
    massearth=5.9722*10**24
    masses=mass #*masssun (natural units)
    return phi,orbitalangle,orbitalradius,eccentricity, masses
    

In [134]:
import random,numpy as np
initdateq=InitialDataEqualMass()

In [135]:
print(initdateq)

(array([0.        , 3.14159265]), array([0., 0.]), array([1., 1.]), array([0., 0.]), array([1., 1.]))


In [136]:
def getxyuv(initdat):
    phi,orbitangle,orbitalradius,eccentricity, masses=initdat
    #print(orbitalradius, phi, np.cos(phi), np.sin(phi))
    metersperAU=149597870700
    Gconstant=6.408*10**-11
    x0=orbitalradius*np.cos(phi)*metersperAU
    y0=orbitalradius*np.sin(phi)*metersperAU
    count=0
    for phi0 in phi:
        if phi0==0:
            print("zero")
            x0[count]=orbitalradius[count]
            y0[count]=0
        if phi0==math.pi:
            print("pi")
            x0[count]=-orbitalradius[count]
            y0[count]=0
        count+=1
    z0=np.zeros(2)
    print(x0)
    print(y0)
    
    #start at perihelion for both (eliptical, doesn't generalize to three body)
    #actually start with circular orbit
    ux0=np.zeros(2) #*149597870700
    #centrepital force balances gravitational force
    reducedmass=np.zeros(2)
    print(masses)
    for i in np.arange(2):
        j=(i+1)%2 #reverse masses
        reducedmass[i]=masses[i]*masses[j]/np.sum(masses)
    print(reducedmass)
    metersperAU=1 #natural units
    rphys=np.zeros(len(masses))
    for i in np.arange(2):
        rphys[i]=orbitalradius[i]*metersperAU*reducedmass[i]/masses[i]
    x0phys=rphys*x0
    y0phys=rphys*y0
    z0phys=np.zeros(2)
    print(rphys)
    #G=1
    Gconstant=1
    F=(Gconstant*reducedmass**2/rphys**2)
    print(F)
    #centF=reducedmass*v**2/rphys
    #centF=accel
    v=np.sqrt(Gconstant*reducedmass/rphys)
    print(v)
    ux0=v*y0
    uy0=v*x0 #initial data in y only
    #evolve in plane only
    #there is a units problem that needs to be fixed
    #velocity initial conditions are not trivial. 
    uz0=np.zeros(2)
    
    #circular orbit   #a=omega^2 * r #v=omega*r #omega=v/r
    omega=v/rphys
    print(omega)
    omegatrue=np.mean(omega) #shouldaccount for numerical effects
    print(omegatrue)
    a=omegatrue**2*rphys
    print("a",a)
    print(phi)
    ax0=-a*x0
    ay0=-a*y0
    az0=np.zeros(2)
    
    
    return reducedmass,x0phys,y0phys,z0phys, ux0, uy0,uz0, ax0, ay0,az0

In [137]:
xyuvaeq=getxyuv(initdateq)
print(xyuvaeq)#In SI units
print(xyuvaeq[1][0])
print(xyuvaeq[0][0]/xyuva[0][1])
print(xyuvaeq)

zero
pi
[ 1. -1.]
[0. 0.]
[1. 1.]
[0.5 0.5]
[0.5 0.5]
[1. 1.]
[1. 1.]
[2. 2.]
2.0
a [2. 2.]
[0.         3.14159265]
(array([0.5, 0.5]), array([ 0.5, -0.5]), array([0., 0.]), array([0., 0.]), array([0., 0.]), array([ 1., -1.]), array([0., 0.]), array([-2.,  2.]), array([-0., -0.]), array([0., 0.]))
0.5
8.3721493136062e-26
(array([0.5, 0.5]), array([ 0.5, -0.5]), array([0., 0.]), array([0., 0.]), array([0., 0.]), array([ 1., -1.]), array([0., 0.]), array([-2.,  2.]), array([-0., -0.]), array([0., 0.]))


# Two Body, Earth and Sun

Generate iniitial data for two body problem, Sun and Earth, using reduced mass framework.

In [74]:
import math
def InitialData():
    random.seed(a=9001)
    
    #initially use an in plane orbit with random starting locations relative to the x axis
    phi=np.array([0,math.pi])
    orbitangle=np.zeros(2)
    #orbitalradius=np.random.uniform(.1,50,2)
    orbitalradius=np.ones(2)
    #start with circular orbits
    #eccentricity=np.random.uniform(0.,.1,2)
    eccentricity=np.zeros(2)
    #magnitude=np.random.uniform(-20,-30,2) #absolute not apparent maginutde
    #magsun=-26.832
    masssun=1.989*10**30
    lsun=3.828*10**26
    massearth=5.9722*10**24
    #luminosity=lsun*10**(0.4*(magnitude-magsun))
    #masses= invertIMF(luminosity,lsun,masssun) #Initial mass function for Main Sequence
    #masses=np.random.uniform(.7,5.) #replace with IMF
    masses=np.array([masssun,massearth])

    return phi,orbitangle,orbitalradius,eccentricity, masses



In [75]:
import random,numpy as np
initdat=InitialData()

In [76]:
print(initdat)

(array([0.        , 3.14159265]), array([0., 0.]), array([1., 1.]), array([0., 0.]), array([1.9890e+30, 5.9722e+24]))


Transform cylindrical initial data to x,y,z, velocity, acceleration initial data

In [77]:
def getxyuv(initdat):
    phi,orbitangle,orbitalradius,eccentricity, masses=initdat
    #print(orbitalradius, phi, np.cos(phi), np.sin(phi))
    metersperAU=149597870700
    Gconstant=6.408*10**-11
    x0=orbitalradius*np.cos(phi)*metersperAU
    y0=orbitalradius*np.sin(phi)*metersperAU
    z0=np.zeros(2)
    

    
    #start at perihelion for both (eliptical, doesn't generalize to three body)
    #actually start with circular orbit
    ux0=np.zeros(2)*149597870700
    #centrepital force balances gravitational force
    reducedmass=np.zeros(2)
    print(masses)
    for i in np.arange(2):
        j=(i+1)%2 #reverse masses
        reducedmass[i]=masses[i]*masses[j]/np.sum(masses)
    print(reducedmass)
    rphys=np.zeros(len(masses))
    for i in np.arange(2):
        rphys[i]=orbitalradius[i]*metersperAU*reducedmass[i]/masses[i]
    x0=rphys*np.cos(phi)
    y0=rphys*np.sin(phi)
    z0=np.zeros(2)
    print(rphys)
    F=(Gconstant*reducedmass**2/rphys**2)
    print(F)
    #centF=reducedmass*v**2/rphys
    #centF=accel
    v=np.sqrt(Gconstant*reducedmass/rphys)
    print(v)
    ux0=v*np.sin(phi)
    uy0=v*np.cos(phi) #initial data in y only
    #evolve in plane only
    #there is a units problem that needs to be fixed
    #velocity initial conditions are not trivial. 
    uz0=np.zeros(2)
    
    #circular orbit   #a=omega^2 * r #v=omega*r #omega=v/r
    omega=v/rphys
    print(omega)
    omegatrue=np.mean(omega) #shouldaccount for numerical effects
    print(omegatrue)
    a=omegatrue**2*rphys
    print("a",a)
    print(phi)
    ax0=-a*np.cos(phi)
    ay0=-a*np.sin(phi)
    az0=np.zeros(2)
    
    
    return reducedmass,x0,y0,z0, ux0, uy0,uz0, ax0, ay0,az0

In [78]:
xyuva=getxyuv(initdat)
print(xyuva)#In SI units
print(xyuva[1][0])
print(xyuva[0][0]/xyuva[0][1])
print(xyuva)

[1.9890e+30 5.9722e+24]
[5.97218207e+24 5.97218207e+24]
[4.49183369e+05 1.49597422e+11]
[1.13276871e+28 1.02126951e+17]
[29188.7795877     50.57847341]
[6.49818796e-02 3.38097227e-10]
0.032490939958166556
a [4.74185445e+02 1.57924190e+08]
[0.         3.14159265]
(array([5.97218207e+24, 5.97218207e+24]), array([ 4.49183369e+05, -1.49597422e+11]), array([0.00000000e+00, 1.83204003e-05]), array([0., 0.]), array([0.00000000e+00, 6.19407656e-15]), array([29188.7795877 ,   -50.57847341]), array([0., 0.]), array([-4.74185445e+02,  1.57924190e+08]), array([-0.00000000e+00, -1.93401354e-08]), array([0., 0.]))
449183.36891987134
1.0
(array([5.97218207e+24, 5.97218207e+24]), array([ 4.49183369e+05, -1.49597422e+11]), array([0.00000000e+00, 1.83204003e-05]), array([0., 0.]), array([0.00000000e+00, 6.19407656e-15]), array([29188.7795877 ,   -50.57847341]), array([0., 0.]), array([-4.74185445e+02,  1.57924190e+08]), array([-0.00000000e+00, -1.93401354e-08]), array([0., 0.]))


Time stepping routine using Euler method and Newtonian differential equation for two body system

In [ ]:
def timestep(step,t,dt,reducedmass,xi,yi,zi, vxi, vyi, vzi, axi, ayi, azi):
    xii=np.zeros(np.size(xi))
    vxii=np.zeros(np.size(vx))
    yii=np.zeros(np.size(yi))
    vyii=np.zeros(np.size(vy))
    zii=np.zeros(np.size(vzi))
    vzii=np.zeros(np.size(vzi))
    rii=np.zeros(np.size(xi))
    axii=axi
    ayii=ayi
    azii=azi
    
    for m in np.arange(len(x)):
        #m represents choices of mass
        i=step
        
        xii[m] = xi[m] + dt*vxi[m]
        #print(xii)
        vxii[m] = vxi[m] + dt*axi[m]
        #print(vxii)
        yii[m]= yi[m] + dt*vyi[m]
        vyii[m] = vyi[m] + dt*ayi[m]
        zii[m]= zi[m] + dt*vzi[m]
        vzii[m] = vzi[m] + dt*azi[m]
        rii[m]=np.sqrt(xi[m]**2+yi[m]**2+zi[m]**2)
    
    
    Gconstant=6.408*10**-11
    for k in np.arange(len(rii)):
        for j in np.arange(len(rii)):
            if j!=k:
                rreljk=np.abs((xi[j] - xi[k])**2+(yi[j]-yi[k])**2+(zi[j]-zi[k])**2)**(1./2.)
                axii[j]+=Gconstant*reducedmass[k]*(xi[j]  - xi[k])/rreljk**3
                ayii[j]+=Gconstant*reducedmass[k]*(yi[j]  - yi[k])/rreljk**3
                azii[j]+=Gconstant*reducedmass[k]*(zi[j]  - zi[k])/rreljk**3
    #print(xii)
    return reducedmass, xii,yii,zii,vxii,vyii,vzii,axii,ayii,azii
                    

Run over several time steps and print

In [ ]:
dt=0.01*31556926 #seconds per year
numsteps=5
mass,x,y,z,vx,vy,vz,ax,ay,az=xyuva
for i in np.arange(1,numsteps):
    t=0.+i*numsteps*dt
    mass, x,y,z,vx,vy,vz,ax,ay,az=timestep(i,t,dt,mass,x,y,z,vx,vy,vz,ax,ay,az)
    print(x,y,vx,vy,ax,ay)
    #print(ay) #forces should be equal and opposite, but in reduced mass framework accelerations are also equal and opposite
    #accelerations should evolve from y to x with time in a sinusoidal manner even in reduced mass framework
    #print(ax)
#mass, x,y,z,vx,vy,vz,ax,ay,az=timestep(2,0,dt,mass,x,y,z,vx,vy,vz,ax,ay,az)
#print(x,y,vx,vy,ax,ay) 
#mass, x,y,z,vx,vy,vz,ax,ay,az=timestep(3,0,dt,mass,x,y,z,vx,vy,vz,ax,ay,az)
#print(x,y,vx,vy,ax,ay) 
#mass, x,y,z,vx,vy,vz,ax,ay,az=timestep(4,0,dt,mass,x,y,z,vx,vy,vz,ax,ay,az)
#print(x,y,vx,vy,ax,ay) 

Object for storing the orbital differential equation and evolving using implicit RK4 using Newtonian differential equation for two body system

In [ ]:
class OrbitDiffEq:
    def __init__(self,reducedmass,x0,y0,z0, ux0, uy0,uz0, ax0, ay0,az0):
        self.reducedmass=reducedmass
        self.xi=x0
        self.yi=y0
        self.zi=z0
        self.vxi=ux0
        self.vyi=uy0
        self.vzi=uz0
        self.axi=ax0
        self.ayi=ay0
        self.azi=az0
    def dxidt(self,t):
        return self.vxi
    def dyidt(self,t):
        return self.vyi
    def dzidt(self,t):
        return self.vzi
    def dvxidt(self,t):
        #return axi[m]
        axii=self.axi
        rii=np.sqrt(self.xi**2+self.yi**2+self.zi**2)
        Gconstant=6.408*10**-11
        for k in np.arange(len(rii)):
            for j in np.arange(len(rii)):
                if j!=k:
                    rreljk=np.abs((self.xi[j] - self.xi[k])**2+(self.yi[j]-self.yi[k])**2+(self.zi[j]-self.zi[k])**2)**(1./2.)
                    axii[j]+=Gconstant*self.reducedmass[k]*(self.xi[j]  - self.xi[k])/rreljk**3
        self.axi=axii
        return axii
    def dvyidt(self,t):
        #return axi[m]
        ayii=self.ayi
        rii=np.sqrt(self.xi**2+self.yi**2+self.zi**2)
        Gconstant=6.408*10**-11
        for k in np.arange(len(rii)):
            for j in np.arange(len(rii)):
                if j!=k:
                    rreljk=np.abs((self.xi[j] - self.xi[k])**2+(self.yi[j]-self.yi[k])**2+(self.zi[j]-self.zi[k])**2)**(1./2.)
                    ayii[j]+=Gconstant*self.reducedmass[k]*(self.yi[j]  - self.yi[k])/rreljk**3
        self.ayi=ayii
        return ayii
    def dvzidt(self,t):
        #return axi[m]
        azii=self.azi
        rii=np.sqrt(self.xi**2+self.yi**2+self.zi**2)
        Gconstant=6.408*10**-11
        for k in np.arange(len(rii)):
            for j in np.arange(len(rii)):
                if j!=k:
                    rreljk=np.abs((self.xi[j] - self.xi[k])**2+(self.yi[j]-self.yi[k])**2+(self.zi[j]-self.zi[k])**2)**(1./2.)
                    azii[j]+=Gconstant*self.reducedmass[k]*(self.zi[j]  - self.zi[k])/rreljk**3
        self.azi=azii
        return azii
    def updateINTERNAL(self,xii,yii,zii,vxii,vyii,vzii):
        self.xi=xii
        self.yi=yii
        self.zi=zii
        self.vxi=vxii
        self.vyi=vyii
        self.vzi=vzii
        return self
    def update(self,xii,yii,zii,vxii,vyii,vzii,axii,ayii,azii):
        self.xi=xii
        self.yi=yii
        self.zi=zii
        self.vxi=vxii
        self.vyi=vyii
        self.vzi=vzii
        self.axi=axii
        self.ayi=ayii
        self.azi=azii
    def print2D(self):
        print(self.xi,self.yi,self.vxi,self.vyi,self.axi,self.ayi)
        return self
    def list2D(self):
        return self.xi,self.yi,self.vxi,self.vyi,self.axi,self.ayi
    def timestepRK4ODE(self,step,t,dt):

    
        h=dt
        #tnew,ynew, intval=RK4(h,t,y,f)
        #m represents choices of mass
        i=step
        
        tnew,intvalx=RK4implicit(h,t,self.dxidt)
        xii = intvalx
        tnew,intvalvx=RK4implicit(h,t,self.dvxidt)
        vxii=intvalvx
        tnew,intvaly=RK4implicit(h,t,self.dyidt)
        yii = intvaly
        tnew,intvalvy=RK4implicit(h,t,self.dvyidt)
        vyii=intvalvy
        tnew,intvalz=RK4implicit(h,t,self.dzidt)
        zii = intvalz
        tnew,intvalvz=RK4implicit(h,t,self.dvzidt)
        vzii=intvalvz
 
        #print(xii)
        self.updateINTERNAL(xii,yii,zii,vxii,vyii,vzii)
        return reducedmass, xii,yii,zii,vxii,vyii,vzii,self.axi,self.ayi,self.azi

Implicit RK4

In [ ]:
def RK4implicit(h,t,f): #not a finite difference so no step in y
    k1= h*f(t)
    k2=h*f(t+h/2.)
    k3=h*f(t+h/2.)
    k4=h*f(t+h)
    return t+h, f(t)+1/6.*(k1+2.*k2+2.*k3+k4)




Initialize iniitial conditions 

In [ ]:
reducedmass,x0,y0,z0, ux0, uy0,uz0, ax0, ay0,az0=xyuva
ODE= OrbitDiffEq(reducedmass,x0,y0,z0, ux0, uy0,uz0, ax0, ay0,az0)
ODE.print2D()

Extract coordinates

In [ ]:
mass,x,y,z,vx,vy,vz,ax,ay,az=xyuva
print(x,y,vx,vy,ax,ay)
print(xyuva)

Evolve the differential equation and print

In [ ]:
dt=0.001*31556926 #seconds per year
numsteps=5
reducedmass,x,y,z,vx,vy,vz,ax,ay,az=xyuva
ODE.print2D()
for i in np.arange(1,numsteps):
    t=0.+i*numsteps*dt
    reducedmass, x,y,z,vx,vy,vz,ax,ay,az=ODE.timestepRK4ODE(i,t,dt)
    ODE.print2D()
    ODE.update(x,y,z,vx,vy,vz,ax,ay,az)



In [ ]:
secperyr=31556926

Evolve the differential equation and store in a pandas data frame for later plotting

In [ ]:
ODE2= OrbitDiffEq(reducedmass,x0,y0,z0, ux0, uy0,uz0, ax0, ay0,az0)



In [ ]:
dt=0.01*31556926 #seconds per year
numsteps=5
reducedmass,x,y,z,vx,vy,vz,ax,ay,az=xyuva
datainit=[[0,0.0,x,y,z,vx,vy,vz,ax,ay,az]]
import pandas as pd
columns =['step','time','x' ,'y','z','vx','vy','vz','ax','ay','az']
df = pd.DataFrame(data=datainit, columns=columns)




In [ ]:
display(df)

In [ ]:
for i in np.arange(1,numsteps):
    t=0.+i*numsteps*dt
    reducedmass, x,y,z,vx,vy,vz,ax,ay,az=ODE2.timestepRK4ODE(i,t,dt)
    df.loc[i]=i,t,x,y,z,vx,vy,vz,ax,ay,az
    ODE2.update(x,y,z,vx,vy,vz,ax,ay,az)


In [ ]:
display(df)

In [ ]:
xser=df['x']
xser
xser[1][0]

Each pandas column has two entries because there are two bodies, the sun and the earth. These need to be split into columns one and two in a new table for each pandas column in the original table. The following routine automates extraction of these entries for each column. 

In [ ]:
def mergeStarDataIntoDataFrame(col):
    rawdataser=df[col]
    numstars=len(rawdataser[0])
    columns=np.arange(numstars)
    numentries=len(rawdataser)
    index=np.arange(numentries)
    dfnew=pd.DataFrame(columns=columns,index=index)
    for coln in np.arange(numstars):
        datacollist=[]
        for row in np.arange(numentries):
            datacollist.append(rawdataser[row][coln])
        dfnew[coln]=datacollist
    return dfnew

In [ ]:
dfx=mergeStarDataIntoDataFrame('x')

In [ ]:
display(dfx)

In [ ]:
dfy=mergeStarDataIntoDataFrame('y')

In [ ]:
display(dfy)

In [ ]:
dfvx=mergeStarDataIntoDataFrame('vx')

In [ ]:
display(dfvx)

In [ ]:
dfvy=mergeStarDataIntoDataFrame('vy')

In [ ]:
display(dfvy)

In [ ]:
dfax=mergeStarDataIntoDataFrame('ax')

In [ ]:
display(dfax)

In [ ]:
dfay=mergeStarDataIntoDataFrame('ay')

In [ ]:
display(dfay)

In [ ]:
df['time'].tolist()

Plots time versus position of Earth, in reduced mass coordinates

In [ ]:
from bokeh.plotting import figure, output_notebook, show
p=figure(plot_width=300,plot_height=300,x_axis_label="Time (s)",y_axis_label="y displacement (m), Earth")
show(p)
p.circle(x=np.array(df['time'].tolist()),y=np.array(dfy[1].tolist()))
show(p)
output_notebook()